In [1]:
%pylab inline
import os
import nrrd
import json

Populating the interactive namespace from numpy and matplotlib


In [2]:
DATAPATH = '/raid/mpsych/CACTAS/DATA/ESUS/'
NEWPATH = '/raid/mpsych/CACTAS/DATA/CLEANED/'
MASKPATH= '/raid/mpsych/CACTAS/DATA/CA24/'

In [3]:
all_slices = []
data = {}
count = 0
ids = []

for f in os.listdir(DATAPATH):
    if f.endswith('img.nrrd'):
        current_id = f.split('.')[0]
        ids += [int(current_id)]
        ids = sorted(ids)

In [4]:
len(ids)

70

In [6]:
for i in range(len(ids)):
    if os.path.isfile(DATAPATH + str(ids[i]) + '.img.nrrd'):
        im, h = nrrd.read(DATAPATH + str(ids[i]) + '.img.nrrd')
    elif os.path.isfile(DATAPATH + str(ids[i]) + '.b.img.nrrd'):
        im, h = nrrd.read(DATAPATH + str(ids[i]) + '.b.img.nrrd')
        
#     print(f"Data type for {ids[i]}: {im.dtype}") 

    data[ids[i]] = count
    
    all_slices = []
    count = 0
    for z in range(im.shape[2]):
        all_slices.append(im[:, :, z])
        count += 1

    a = np.zeros((len(all_slices), all_slices[0].shape[0], all_slices[0].shape[1]), dtype=all_slices[0].dtype)

    for j, s in enumerate(all_slices):
        a[j] = s

#     images_file = os.path.join(NEWPATH, 'images/' + str(ids[i]) + '.image.npy')
#     np.save(images_file, a)

In [9]:
for i in range(len(ids)):
    if os.path.isfile(DATAPATH + str(ids[i]) + '.b.seg.nrrd'):
        im, h = nrrd.read(DATAPATH + str(ids[i]) + '.b.seg.nrrd')
    
    data[ids[i]] = count
    
    #print(f"Data type for {ids[i]}: {im.dtype}") 

    all_slices = []
    count = 0
    for z in range(im.shape[2]):
        all_slices.append(im[:, :, z])
        count += 1
    
    a = np.zeros((len(all_slices), all_slices[0].shape[0], all_slices[0].shape[1]), dtype=all_slices[0].dtype)

    for j, s in enumerate(all_slices):
        a[j] = s

#     images_file = os.path.join(NEWPATH, 'labels/' + str(ids[i]) + '.seg.npy')
#     np.save(images_file, a)

IndentationError: unexpected indent (3705317097.py, line 8)

In [8]:
#images = np.load(NEWPATH +'images/61.image.npy')
#labels = np.load(NEWPATH + 'labels/61.seg.npy')
#masks = np.load(NEWPATH + 'masks/61.ca.seg.npy')

In [11]:
#images.shape, labels.shape, masks.shape

In [12]:
# for i in range(images.shape[0]):
#     plt.imshow(images[i,:,:])
#     plt.imshow(labels[i,:,:], cmap='gray', alpha=0.5)
#     plt.show()

In [8]:
for i in range(len(ids)):
    if os.path.isfile(MASKPATH + str(ids[i]) + '.ca.seg.nrrd'):
        im, h = nrrd.read(MASKPATH + str(ids[i]) + '.ca.seg.nrrd')

    data[ids[i]] = count
    
    print(f"Data type for {ids[i]}: {im.dtype}") 

    all_slices = []
    count = 0
    for z in range(im.shape[2]):
        all_slices.append(im[:, :, z])
        count += 1
    
    a = np.zeros((len(all_slices), all_slices[0].shape[0], all_slices[0].shape[1]), dtype=all_slices[0].dtype)

    for j, s in enumerate(all_slices):
        a[j] = s

#     images_file = os.path.join(NEWPATH, 'masks/' + str(ids[i]) + '.ca.seg.npy')
#     np.save(images_file, a)


IndentationError: unexpected indent (3716850485.py, line 8)

In [36]:
# images = np.load(NEWPATH +'images/11.image.npy')
# masks = np.load(NEWPATH + 'masks/11.ca.seg.npy')

In [37]:
# for i in range(images.shape[0]):
#     plt.imshow(images[i,:,:])
#     plt.imshow(masks[i,:,:], cmap='gray', alpha=0.5)
#     plt.show()

In [5]:
patients = {}

for file in os.listdir(NEWPATH + 'images/'):
    #print("images: " + file)
    if file.endswith(".image.npy"):
        patient_id = int(file.split('.')[0])
        if patient_id not in patients:
            patients[patient_id] = {}
        patients[patient_id]['image'] = np.load(os.path.join(NEWPATH + 'images/', file))

for file in os.listdir(NEWPATH + 'labels/'):
    #print("labels: " + file)
    if file.endswith(".seg.npy"):
        patient_id = int(file.split('.')[0])
        if patient_id in patients:
            patients[patient_id]['label'] = np.load(os.path.join(NEWPATH + 'labels/', file))

for file in os.listdir(NEWPATH + 'masks/'):
    #print("masks: " + file)
    if file.endswith(".ca.seg.npy"):
        patient_id = int(file.split('.')[0])
        if patient_id in patients:
            patients[patient_id]['mask'] = np.load(os.path.join(NEWPATH + 'masks/', file))

In [6]:
for patient_id, data in patients.items():
    if 'label' in data:
        images = data.get('image')
        labels = data.get('label')
        masks = data.get('mask')
        
        remove_list = [i for i, label in enumerate(labels) if label.max() == 0]
        
        if images is not None:
            #remove_list = [i for i, label in enumerate(labels) if label.max() == 0]
            data['image'] = [image for i, image in enumerate(images) if i not in remove_list]
            data['label'] = [label for i, label in enumerate(labels) if i not in remove_list]
            data['mask'] = [mask for i, mask in enumerate(masks) if i not in remove_list]

In [7]:
for patient_id, data in patients.items():
    if data is not None:
        np.save(os.path.join(NEWPATH + 'filt_img/', f'{patient_id}.image.npy'), data['image'])
        np.save(os.path.join(NEWPATH + 'filt_lb/', f'{patient_id}.seg.npy'), data['label'])
        np.save(os.path.join(NEWPATH + 'filt_msk/', f'{patient_id}.ca.seg.npy'), data['mask'])

In [27]:
# images_up = np.load(NEWPATH +'filtered_images/10.image.npy')
# labels_up = np.load(NEWPATH + 'filtered_labels/10.seg.npy')
# masks_up = np.load(NEWPATH + 'filtered_masks/10.ca.seg.npy')

In [30]:
# for i in range(images_up.shape[0]):
#     plt.imshow(images_up[i,:,:])
#     #plt.imshow(labels_up[i,:,:], cmap='gray', alpha=0.5)
#     plt.imshow(masks_up[i,:,:], cmap='jet', alpha=0.5)
#     plt.show()

In [8]:
import shutil

In [13]:
# Define the source directory containing the files
source_directory = NEWPATH + 'filt_msk/'

# Define the train and test directories
train_directory = NEWPATH + 'filt_msk/train'
test_directory = NEWPATH + 'filt_msk/test'

# Get the list of files in the source directory
files = [file for file in os.listdir(source_directory) if file.endswith(".npy")]

files.sort()

In [14]:
# Copy the first 42 files to the train directory
for file_name in files[:42]:
    source_path = os.path.join(source_directory, file_name)
    destination_path = os.path.join(train_directory, file_name)
    shutil.copy(source_path, destination_path)

# Copy the remaining files to the test directory
for file_name in files[42:]:
    source_path = os.path.join(source_directory, file_name)
    destination_path = os.path.join(test_directory, file_name)
    shutil.copy(source_path, destination_path)

In [3]:
# train_images_directory = NEWPATH + 'filtered_images/train'
# test_images_directory = NEWPATH + 'filtered_images/test'
# train_labels_directory = NEWPATH + 'filtered_labels/train'
# test_labels_directory = NEWPATH + 'filtered_labels/test'
# train_masks_directory = NEWPATH + 'filtered_masks/train'
# test_masks_directory = NEWPATH + 'filtered_masks/test'

In [24]:
# images_up = np.load(NEWPATH +'filtered_images/61.image.npy')
# labels_up = np.load(NEWPATH + 'filtered_labels/61.seg.npy')
# masks_up = np.load(NEWPATH + 'filtered_masks/61.ca.seg.npy')

In [1]:
# images_up.shape, labels_up.shape, masks_up.shape